# **<center> Análisis de Sentimientos de los candidatos presidenciales </br> Ecuador 2021, en la red social Twitter </center>**
<h6>
    <center> Miguel Angel Macias, Jonnathan Campoberde <br/> 
    <i>Universidad de Cuenca <br/>  <i>Facultad de Ingenieria - Escuela de Sistemas<br/> Cuenca, Ecuador</i>
    <br/> { mangel.maciasn, jonnathan.campoberde }@ucuenca.edu.ec</center>
<h6>

# **I.&emsp;Introducción**
El presente trabajo tiene como objetivo emplear de manera práctica todos los conocimientos adquiridos en el transcurso de la asignatura “Text Mining”. Se trabajará con un dataset... . Se busca analizar el ..., detectando las ... por medio de ... . Todo esto con el propósito de ... .

## **1.1.&emsp;Objetivos del Proyecto**
>El objetivo del proyecto es el Análisis de Sentimientos de los candidatos presidenciales de Ecuador 2021, para luego contrastar los resultados obtenidos con los resultados oficiales de las elecciones Posteriormente se desea responder a las preguntas definidas en 1.2.

## **1.2.&emsp;Preguntas de investigación**
>- RQ1:     ¿Qué tan precisamente los datos de Twitter reflejan el sentimiento político en un proceso electoral?
>- RQ1:     ¿En qué grado de aceptación se encuentran los candidatos dentro de la red Social Twitter?
>- RQ3:     ¿Los resultadso oficiales son parecidos a los hallados en nuestro análisis?


# **II.&emsp;TEXT CLEANING**

## **2.1.&emsp; Reading dataset**
>Para este paso...

In [1]:
#!pip install emoji

In [2]:
import warnings
warnings.filterwarnings("ignore")

#from datetime import datetime, timezone
import pytz
import numpy as np              # Para operaciones matematicas
import pandas as pd             # Manejo de datos / dataframes
import re
import emoji
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer
%matplotlib inline

In [3]:
data_raw = '../data/raw'              # directorio que contiene toda la data sin manipular
data_processed = '../data/processed'  # directorio que contiene data procesada

In [4]:
df_arauz = pd.read_csv(data_raw+"/all_tweets_arauz.csv")
df_arauz['created_at'] = pd.to_datetime(df_arauz['created_at'], utc=pytz.UTC)
df_arauz['user.created_at'] = pd.to_datetime(df_arauz['user.created_at'], utc=pytz.UTC)
df_arauz

,created_at,favorite_count,full_text,id_str,metadata.iso_language_code,retweeted_status.created_at,retweeted_status.full_text,user.created_at,user.favourites_count,user.followers_count,user.friends_count,user.id_str,user.location,user.verified,retweet_count
0,2021-02-01 00:00:00+00:00,1,#Elecciones2021Ec | ¿Las propuestas económicas...,1356029498307784705,es,NaN,NaN,2011-12-05 13:59:33+00:00,269,12756,288,429043466,Ecuador,False,0
1,2021-01-31 23:59:54+00:00,0,RT @carlosrojasecu: El candidato Andrés Arauz ...,1356029470554128384,es,Sun Jan 31 15:51:12 +0000 2021,El candidato Andrés Arauz @ecuarauz no aceptó ...,2010-04-22 06:20:32+00:00,15211,1643,3233,135774682,Ombligo del mundo,False,154
2,2021-01-31 23:59:53+00:00,0,"RT @VTVcanal8: Candidato del Correísmo, Andrés...",1356029467274194946,es,Sun Jan 31 19:20:13 +0000 2021,"Candidato del Correísmo, Andrés Arauz aumentó ...",2020-11-24 03:29:03+00:00,10157,169,163,1331077287144517634,NaN,False,387
3,2021-01-31 23:59:48+00:00,0,"RT @VTVcanal8: Candidato del Correísmo, Andrés...",1356029444817776641,es,Sun Jan 31 19:20:13 +0000 2021,"Candidato del Correísmo, Andrés Arauz aumentó ...",2020-09-02 18:22:54+00:00,17,270,247,1301224047850721281,venezuela caracas,False,387
4,2021-01-31 23:59:30+00:00,0,RT @ecuadorenvivo: #ELN habría prestado $80.00...,1356029369324593155,es,Sun Jan 31 16:18:52 +0000 2021,#ELN habría prestado $80.000 para la campaña d...,2010-08-20 20:40:04+00:00,2653,197,226,180914917,Quito,False,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133334,2021-02-01 00:01:08+00:00,0,RT @politikcorrecta: El candidato por @cendemo...,1356029780018294787,es,Sun Jan 31 15:31:00 +0000 2021,"El candidato por @cendemocratico, Andrés Arauz...",2017-10-02 00:43:11+00:00,125237,307,740,914651941912883201,"Pichincha, Ecuador",False,270
133335,2021-02-01 00:00:44+00:00,0,RT @CarlosP05340289: Pelearse con tus amigos p...,1356029680277741571,es,Sun Jan 31 22:05:36 +0000 2021,Pelearse con tus amigos por política es lo más...,2010-03-18 23:52:39+00:00,9129,395,839,124306327,Switzerland,False,191
133336,2021-02-01 00:00:23+00:00,0,"RT @ErmelRosario: 🇪🇨POR MI LINDO ECUADOR, SOLO...",1356029594864934916,es,Sun Jan 31 15:05:16 +0000 2021,"🇪🇨POR MI LINDO ECUADOR, SOLO 1 VUELTA, VOTA TO...",2013-09-16 19:39:40+00:00,26502,403,496,1872797210,NaN,False,9
133337,2021-02-01 00:00:21+00:00,0,RT @petrogustavo: Intentan por todos los medio...,1356029583942971395,es,Sat Jan 30 10:12:43 +0000 2021,Intentan por todos los medios impedir la victo...,2012-06-16 17:39:50+00:00,26931,232,38,610235412,NaN,False,1557


## **2.2.&emsp; Removing Duplicates**
>lorem ipsum

In [5]:
# Count duplicates
len(df_arauz['full_text'])-len(df_arauz['full_text'].drop_duplicates())

121032

In [6]:
df_arauz_processed = df_arauz.drop_duplicates( ['full_text'], keep='first')

In [7]:
len(df_arauz_processed['full_text'])

12307

## **2.3.&emsp; Identify tweets by trolls**
>Lorem ipsum

In [8]:
def is_troll(Dt, Dc, Fc, fc, v):
    '''
    ----------------------------------
    | Variable Description           |
    ----------------------------------
    | Dt  stands for Date Tweet      |
    | Dc  stands for Date creation   |
    | Fc  stands for Followers count |
    | fc  stands for friend count    |
    | v   stands for verified        |
    ----------------------------------
    | Weight penalization            |
    ----------------------------------
    | Timeliness metric:  WD = 0.7   |
    | Followers metric:   WF = 0.2   |
    | Following metric:   Wf = 0.1   |
    ----------------------------------
    | Acceptance Threshold           |
    ----------------------------------
    | Non Troll   > 0.8              |
    | Troll       < 0.8              |
    ----------------------------------
    '''
    # If user is verified then is a real person
    if v:
        return True
    else:

        threshold = 0.7*((Dt-Dc).days)/180 + 0.2*(Fc/250) + 0.1*(fc/300)

        return threshold < 0.8


In [9]:
df_arauz_processed['is_troll'] = df_arauz_processed.apply(lambda x: is_troll(x['created_at'],
                                                         x['user.created_at'],
                                                         x['user.followers_count'],
                                                         x['user.friends_count'],
                                                         x['user.verified']), axis=1)
# Se excluyen tweets trolls del dataset
#df_arauz_clean = df_arauz_processed[df_arauz_processed['is_troll']==False]

## **2.4.&emsp; Removing characters and Specific Stopwords**
>lorem ipsum

In [10]:
def clean_text(text):
    """Function to clean text extracted from the Internet with html tags
    
    Parameters
    ----------
    text: str 
      original string
    
    Return
    ------
    text: str
      modified initial string
    """

    text = text.lower() # lowercase text
    text = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", text) #Remove http links
    text = EMOJI.sub(r'', text)
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text

In [11]:
# filtrado de caracteres y stopwords
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@#:,;¿?!¡-]')
EMOJI = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)

STOPWORDS = ('rt')  # Adding new stopwords to the set, 'cause theese words repeat in all docs

df_arauz_processed['full_text'] = df_arauz_processed["full_text"].apply(clean_text)

### _Luego de remover los caracteres especiales pueden existir Tweets vacios, debido a que estos solo contenian emotes o enlaces en su interior. Es por esta razon que se buscan estos espacios en blanco y se los quita_

In [12]:
df_arauz_processed[df_arauz_processed['full_text']==""]["full_text"].count()

42

In [13]:
df_arauz_processed.drop(df_arauz_processed.loc[df_arauz_processed["full_text"]==""].index, inplace=True)

In [14]:
df_arauz_processed[df_arauz_processed['full_text']==""]["full_text"].count()

0

In [15]:
df_arauz_processed.to_csv(data_processed+'/arauz_clean.csv', 
                        index=False, header=True, chunksize=10000)